In [4]:
"""
Created on Wed Oct  3 14:47:19 2018

@author: Jessie
"""
%matplotlib inline
from __future__ import print_function
import pandas as pd
import numpy as np
import scipy.stats as stats
import sklearn
import random
import os
from sklearn.linear_model import *
import matplotlib.pyplot as plt
from IPython.display import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
#import statsmodels.api as sm
#import statsmodels.formula.api as smf
#import statsmodels.stats.api as sms
from pathlib import Path

In [5]:
#import numpy array
def LoadSplitData(test_size):
    DATA_DIR = "Data"
    ENCODING_DIR = os.path.join(DATA_DIR, "binEncoding.csv")
    data_xy = pd.read_csv(ENCODING_DIR)
    X = data_xy.drop(data_xy.columns[0:8], axis=1)
    y = data_xy[data_xy.columns[1]]
    
    
    #Split data into test/train set 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
    DummyX_train, DummyX_test, Dummyy_train, Dummyy_test = train_test_split(data_xy, y, test_size=0.30, random_state=42)
    
    return X, y, X_train, X_test, y_train, y_test, DummyX_train, DummyX_test, Dummyy_train, Dummyy_test
    


In [6]:
DATA_DIR = "Data"
ENCODING_DIR = os.path.join(DATA_DIR, "binEncoding.csv")

data_xy = pd.read_csv(ENCODING_DIR)
X = data_xy.drop(data_xy.columns[0:8], axis=1)
Y = data_xy[data_xy.columns[1]]
Y.head()    

0    1
1    1
2    1
3    1
4    1
Name: price_delta>.3, dtype: int64

In [7]:
X, y, X_train, X_test, y_train, y_test, DummyX_train, DummyX_test, Dummyy_train, Dummyy_test = LoadSplitData(0.30) #Decide the test size (% of total data)

In [8]:
#Check Data
X_train.head()
print(X_train.shape)
print(DummyX_train.shape)

(1174, 250)
(1174, 258)


In [9]:
#Use Logistic Regression - Testing with dummy-y-variable

def DummyTest(DummyX_train, Dummyy_train):
    logReg = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, random_state=0, solver='liblinear')

    #Test with dummy 
    clfDummy = logReg.fit(DummyX_train, Dummyy_train)

    #predict on all rows=articles from row 2 onwards
    pred_train = clfDummy.predict(DummyX_train.values[:2,:]) 

    Accuracy = clfDummy.score(DummyX_train,Dummyy_train) #Returns the mean accuracy on the given test data and labels (should be 1.0)
    #print first 10 predictions
    #print(pred_train[0:10]) 
    return Accuracy

In [10]:
print(DummyTest(DummyX_train, Dummyy_train))


1.0


In [11]:
#Predicting on Real DataSet - Returnds mean accuracy and CI @95%
#Test with dummy 
logReg = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, random_state=0, solver='liblinear')
clf = logReg.fit(X_train, y_train)

#predict on all rows=articles from row 2 onwards
pred_train = clf.predict(X_train.values[:2,:]) 
pred_test = clf.predict(X_test.values[:2,:])

In [12]:
#Without Dummy Variable
TrainScores = clf.score(X_train,y_train)
TestScores = clf.score(X_test,y_test)

#Return the mean accuracy on the given test data and labels (should be 1.0)
print(TrainScores)
print(TestScores)

0.7163543441226575
0.5535714285714286


In [45]:
def articleRanking(X):
    ## Import Article Data
    #input whatever filename contains article info. Right now, that file is sitting in the DataCollection Rep
    file_name = 'retailarticles-18-11-06.xlsx'
    thispath = Path().absolute()
    ARTICLE_DIR = os.path.join(thispath.parent, "DataCollection", "Scripts", "Data", file_name)
    arts = pd.read_excel(ARTICLE_DIR)
    
    probs = clf.predict_proba(X)
    
    probs_df = pd.DataFrame(probs)
    probs_df = probs_df.drop([0], axis=1)
    probs_df['title'] = pd.DataFrame(arts['title'])
    probs_df['classification'] = clf.predict(X)

    #probs_df.head()
    probs_df = probs_df.sort_values(by=[1], ascending=False)
    return probs_df

In [46]:
probs_df = articleRanking(X)
probs_df.head()


,1,title,classification
1614,0.995848,"Mall stocks rally on holiday sales optimism, a...",1
289,0.993670,Walgreens unveils rebranding effort as pharmac...,1
254,0.992801,Why Albertsons Is Buying a Big Chunk of Rite Aid,1
1120,0.991787,Amazon Is Defined by Billions and Millions; Me...,1
1603,0.991304,Walmart to acquire plus-size online retailer E...,1


In [18]:
def random_mean_ci(X, y, num_tests):
    # train_results is a list of train accuracy results for the differrent random splits of the dataset
    train_results = []
    
    # test_results is a list of test accuracy results for the differrent random splits of the dataset
    test_results = []
    
    # Write your code here
    for i in range(num_tests):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random.randint(1,1000))

        # train a logistic regression classifier
        clf = LogisticRegression(C=1.0, penalty='l2', dual=False, tol=0.0001, fit_intercept=True, solver='liblinear').fit(X_train, y_train)

        # predict on train and test set
        y_train_predict = clf.predict(X_train)
        y_test_predict = clf.predict(X_test)

        # calculate train and test accuracy
        train_accuracy = accuracy_score(y_train, y_train_predict)
        test_accuracy = accuracy_score(y_test, y_test_predict)

        # report results, update the results array
        train_results.append(train_accuracy)
        test_results.append(test_accuracy)

    # calculate the train mean and the 95% confidence interval for the list of results
    train_mean = np.mean(train_results)
    train_ci_low, train_ci_high = stats.t.interval(0.95, len(train_results)-1, loc=train_mean, scale=stats.sem(train_results))

    # calculate the test mean and the 95% confidence interval for the list of results
    test_mean = np.mean(test_results)
    test_ci_low, test_ci_high = stats.t.interval(0.95, len(test_results)-1, loc=test_mean, scale=stats.sem(test_results))
    
    # validate return types
    assert isinstance(train_mean, float) and isinstance(train_ci_low, float) and isinstance(train_ci_high, float), "return types"
    assert isinstance(test_mean, float) and isinstance(test_ci_low, float) and isinstance(test_ci_high, float), "return types"
    
    return train_mean, train_ci_low, train_ci_high, test_mean, test_ci_low, test_ci_high

In [19]:
#Accuracy and CI over 10 runs
train_mean, train_low, train_high, test_mean, test_low, test_high = random_mean_ci(X, y, num_tests = 10)
print("Train mean accuracy over 10 random splits: {}".format(train_mean))
print("Train confidence interval over 10 random splits: [{}, {}]".format(train_low, train_high))
print("Test mean accuracy over 10 random splits: {}".format(test_mean))
print("Test confidence interval over 10 random splits: [{}, {}]".format(test_low, test_high))

Train mean accuracy over 10 random splits: 0.7402896081771722
Train confidence interval over 10 random splits: [0.7288090750212393, 0.751770141333105]
Test mean accuracy over 10 random splits: 0.5650793650793651
Test confidence interval over 10 random splits: [0.547570102462968, 0.5825886276957621]


An initial run with Retail only Articles, to determine marking moving or not: 

Train mean accuracy over 10 random splits: 0.7402896081771722
Train confidence interval over 10 random splits: [0.7288090750212393, 0.751770141333105]
Test mean accuracy over 10 random splits: 0.5650793650793651
Test confidence interval over 10 random splits: [0.547570102462968, 0.5825886276957621]
